In [1]:
import tkinter as tk
import cv2
from PIL import Image, ImageTk
import numpy as np
window = tk.Tk()  #Makes main window
window.config(background="#FFFFFF")

#Graphics window
imageFrame = tk.Frame(window, width=600, height=500)
imageFrame.grid(row=0, column=0, padx=10, pady=2)

#Capture video frames
cap = cv2.VideoCapture(0)

display1 = tk.Label(imageFrame)
display1.pack()

#main function which feed the frames into functions and receives final result    
def show_frame():
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    if act==0:
        cv2image = coloring(cv2image);
    else:
        cv2image = surprise(cv2image);
    img = Image.fromarray(cv2image)            
    imgtk = ImageTk.PhotoImage(image=img)
    display1.imgtk = imgtk
    display1.configure(image=imgtk)
    window.after(10, show_frame)



#function that shows RGB values of pixel after clicking on it
def callback(event):
    global panel
    global act
    if panel:
        panel.destroy()
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    if act==0:
        cv2image = coloring(cv2image);
    else:
        cv2image = surprise(cv2image);
    img = Image.fromarray(cv2image)
    panel = tk.Label(imageFrame,text=img.getpixel((event.x, event.y)))
    panel.pack()

# defining sliders which control shape of color in RGB space
w1 = tk.Scale(imageFrame, from_=0, to=127,orient=tk.HORIZONTAL) #Half of side along red axis
w1.set(0)
w1.pack(side=tk.LEFT)
w2 = tk.Scale(imageFrame, from_=0, to=127,orient=tk.HORIZONTAL) #Half of side along green axis
w2.set(0)
w2.pack(side=tk.LEFT)
w3 = tk.Scale(imageFrame, from_=0, to=127,orient=tk.HORIZONTAL) #Half of size along blue axis
w3.set(0)
w3.pack(side=tk.LEFT)
w4 = tk.Scale(imageFrame, from_=0, to=255,orient=tk.HORIZONTAL) #Center of side along red axis
w4.set(0)
w4.pack(side=tk.LEFT)
w5 = tk.Scale(imageFrame, from_=0, to=255,orient=tk.HORIZONTAL) #Center of side along green axis
w5.set(0)
w5.pack(side=tk.LEFT)
w6 = tk.Scale(imageFrame, from_=0, to=255,orient=tk.HORIZONTAL) #Center of size along blue axis
w6.set(0)
w6.pack(side=tk.LEFT)

#Function defines the shape of color and changes an image(frame) according to it
def coloring(image):
    image1=cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    image2=cv2.cvtColor(image1,cv2.COLOR_GRAY2RGB)
    rr=w1.get()
    gr=w2.get()
    br=w3.get()
    rc=w4.get()
    gc=w5.get()
    bc=w6.get()
    lower = np.array([rc-rr, gc-gr, bc-br])
    upper = np.array([rc+rr, gc+gr, bc+br])
    for w in lower:
        if w<0:
            w=0
    for w in upper:
        if w>255:
            w=255
    mask = cv2.inRange(image, lower, upper)
    output = cv2.bitwise_and(image, image, mask = mask)
    output2 = cv2.bitwise_and(image2, image2, mask = cv2.bitwise_not(mask))
    output = cv2.bitwise_or(output,output2)
    return output

#Changes the value of global variable act after pressing the button
def actCh():
    global act
    act = 1-act
    
#surprise button implementation, works when global variable act equal to 1
def surprise(image):
    num_down=2
    num_bilateral=7
    img_color = image
    for _ in range(num_down):
        img_color=cv2.pyrDown(img_color)
    for _ in range(num_bilateral):
        img_color=cv2.bilateralFilter(img_color,d=5,sigmaColor = 10,sigmaSpace=7)
    for _ in range(num_down):
        img_color=cv2.pyrUp(img_color)
    img_gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    img_blur = cv2.medianBlur(img_gray,7)
    img_edge = cv2.adaptiveThreshold(img_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,
                                    blockSize=11,C=2)
    img_edge = cv2.cvtColor(img_edge,cv2.COLOR_GRAY2RGB)
    img_surprise = cv2.bitwise_and(img_color,img_edge)
    return img_surprise



act = 0
panel=None
tk.Button(imageFrame, text='Surprise', command=actCh).pack()
show_frame()
cv2.waitKey(10)
display1.bind("<Button-1>", callback)
display1.pack()
window.mainloop()



